In [1]:
# import os
# import json
# from googletrans import Translator  # Install using: pip install googletrans==4.0.0-rc1

# def translate_json_files(input_folder, output_folder, target_language):
#     """
#     Reads JSON files from the input folder, translates their content, 
#     and writes them to the output folder.
#     """
#     os.makedirs(output_folder, exist_ok=True)
    
#     translator = Translator()

#     def translate_nested_json(data):
#         if isinstance(data, dict):
#             return {key: translate_nested_json(value) for key, value in data.items()}
#         elif isinstance(data, list):
#             return [translate_nested_json(item) for item in data]
#         elif isinstance(data, str):
#             try:
#                 translated = translator.translate(data, dest=target_language)
#                 return translated.text
#             except Exception as e:
#                 print(f"Error translating '{data}': {e}")
#                 return data   
#         else:
#             return data  
        
#     def translate_text(text):
#         """
#         Translate a text string to the target language.
#         """
#         try:
#             # Translate the string to the target language
#             translated = translator.translate(text, dest=target_language)
#             return translated.text
#         except Exception as e:
#             print(f"Error translating '{text}': {e}")
#             return text  # Return the original text if translation fails


#     for filename in os.listdir(input_folder):
#         if filename.endswith(".md"):
#             print('0000 filename', filename)
#             input_path = os.path.join(input_folder, filename)
#             output_path = os.path.join(output_folder, filename)

#             with open(input_path, 'r', encoding='utf-8') as f:
#                 data = f.read()

                
#             print('1111 data', data)


#             translated_data = translate_text(data)


#             print('2222 translated_data', translated_data)

#             with open(output_path, 'w', encoding='utf-8') as f:
#                     f.write(translated_data)
            
#             print(f"Translated and saved: {output_path}")

# # Example usage
# input_folder = "/Users/rakesh.c/rakesh/my/am/inayam/inayam-doc/ai/en"
# output_folder = "/Users/rakesh.c/rakesh/my/am/inayam/inayam-doc/ai/ta"
# target_language = "ta" 

# translate_json_files(input_folder, output_folder, target_language)

#OLD


In [ ]:
import os
import re
from googletrans import Translator
from concurrent.futures import ThreadPoolExecutor, as_completed

def translate_json_files(input_folder, output_folder, target_language):
    os.makedirs(output_folder, exist_ok=True)
    translator = Translator()
    print('translator', translator)

    def translate_text(text, filename, chunk_size=4000):
        try:
            # Split the text into chunks for Google Translate
            chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]
            translated_chunks = [translator.translate(chunk, dest=target_language).text for chunk in chunks]
            return " ".join(translated_chunks)
        except Exception as e:
            print(f"Error translating '{target_language}' '{filename}': {e}")
            return None

    def preserve_markdown_formatting(text):
        """
        This function replaces parts of the markdown text with placeholders,
        translates the text, and then puts everything back together.
        """
        # Define regular expressions for Markdown elements to preserve
        patterns = {
            # 'bold': r'(\*\*.*?\*\*)',  # Bold: **text**
            # 'italic': r'(\*.*?\*)',    # Italic: *text*
            # 'code': r'(`.*?`)',         # Inline code: `code`
            # 'links': r'(\[.*?\]\(.*?\))',  # Links: [text](url)
            # 'headers': r'(#{1,6}\s.*)',  # Headers: # header
            # 'inline_math': r'(\(.*?\))',  # Inline math: \( equation \)
            'block_math': r'(\$\$.*?\$\$)',  # Block math: $$ equation $$
        }

        # Find all Markdown elements and replace them with placeholders
        placeholders = {}
        for key, pattern in patterns.items():
            matches = re.findall(pattern, text)
            for i, match in enumerate(matches):
                placeholder = f"__{key}_{i}__"
                placeholders[placeholder] = match
                text = text.replace(match, placeholder)

        # Translate the remaining plain text
        translated_text = translate_text(text, "markdown_file")

        # Replace the placeholders with the original Markdown elements
        if translated_text:
            for placeholder, original in placeholders.items():
                translated_text = translated_text.replace(placeholder, original)
        
        return translated_text

    def translate_file(filename):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)
        
        with open(input_path, 'r', encoding='utf-8') as f:
            data = f.read()
        
        print(f"Translating {filename}")
        
        # Preserve Markdown formatting (including MathJax) and translate
        translated_data = preserve_markdown_formatting(data)
        
        if translated_data:
            with open(output_path, 'w', encoding='utf-8') as f:
                f.write(translated_data)
            print(f"{filename} translated and saved to {output_path}")
        else:
            print(f"Failed to translate {filename}")

    files_to_translate = [filename for filename in os.listdir(input_folder) if filename.endswith(".md")]

    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = []
        
        # Submit tasks for each file
        for filename in files_to_translate:
            future = executor.submit(translate_file, filename)
            futures.append(future)
        
        # Process completed futures as they finish
        for future in as_completed(futures):
            try:
                future.result()  # Retrieve result or catch exceptions
            except Exception as e:
                print(f"An error occurred: {e}")

# Example usage
# input_folder = "/path/to/input/folder"
# output_folder = "/path/to/output/folder"
# target_language = "es"  # For Spanish
# translate_json_files(input_folder, output_folder, target_language)


In [ ]:
import os

locales = ['ta', 'hi', 'es', 'de', 'ru', 'ja', 'fr', 'pt', 'zh-CN', 'ar', 'it', 'ko', 'te', 'kn']
locales = ['ta']
base_folder = "/Users/rakesh.c/rakesh/my/am/inayam/inayam-doc/unit-converter"
input_base_folder = os.path.join(base_folder, 'en')

# Function to process files
def process_file(input_file_path, output_folder_path, target_language):
    # Add your code to handle translation here
    translate_json_files(input_file_path, output_folder_path, target_language)

# Function to recursively navigate through directories
def process_folders(input_folder, output_folder, locale):
    for root, dirs, files in os.walk(input_folder):
        for file in files:
            input_file_path = os.path.join(root, file)
            # Create corresponding output folder if not exists
            relative_path = os.path.relpath(input_file_path, input_folder)
            output_file_path = os.path.join(output_folder, relative_path)
            os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
            # Process each file
            process_file(input_file_path, output_file_path, locale)

for locale in locales:
    print(f'Started processing locale: {locale}')

    localefolder = 'zh' if locale == 'zh-CN' else locale
    # Process folders inside the 'en' directory
    for input_folder in os.listdir(input_base_folder):
        # Skip 'soldiers' folder (or any folder you wish to exclude)
        if input_folder.lower() in ['soldiers', 'other_folder_to_skip']:
            print(f"Skipping folder: {input_folder}")
            continue

        
        input_folder_path = os.path.join(input_base_folder, input_folder)
        
        # Only process directories inside 'en'
        if os.path.isdir(input_folder_path):
            output_folder_path = os.path.join(base_folder, f'{localefolder}/{input_folder}')
            target_language = locale
            translate_json_files(input_folder_path, output_folder_path, target_language)

    
    print(f'Finished processing locale: {locale}')


started  ta
translator <googletrans.client.Translator object at 0x1058c3820>
Translating octant.md
Translating degree_minute_second.md
Translating small_angle.md
Translating circular_radian.md
Translating radian.md
Translating three_eighth_circle.md
Translating millidegree.md
Translating second_of_arc.md
Translating third_circle.md
Translating quarter_circle.md
three_eighth_circle.mdTranslated and saved: /Users/rakesh.c/rakesh/my/am/inayam/inayam-doc/unit-converter/ta/angle/three_eighth_circle.md
octant.mdTranslated and saved: /Users/rakesh.c/rakesh/my/am/inayam/inayam-doc/unit-converter/ta/angle/octant.md
circular_radian.mdTranslated and saved: /Users/rakesh.c/rakesh/my/am/inayam/inayam-doc/unit-converter/ta/angle/circular_radian.md
Translating turn.md
Translating gradian.md
second_of_arc.mdTranslated and saved: /Users/rakesh.c/rakesh/my/am/inayam/inayam-doc/unit-converter/ta/angle/second_of_arc.md
Translating large_angle.md
small_angle.mdTranslated and saved: /Users/rakesh.c/rakesh/m